In [2]:
import os,sys
import matplotlib.pyplot as plt
#import pandas as pd  # AC got a 'no module named pandas' error.  Do we need it?
import numpy as np
import math
import csv
import scipy

# Eventually we'll have to find a common pathway, for now just change if this is commented out.
#sys.path.insert(0,'/Users/dtn1/gsas2full/GSASII/') # needed to "find" GSAS-II modules
#sys.path.insert(0,'/Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/') # needed to "find" GSAS-II modules

#import GSASIIscriptable as G2sc

## Reading in the table directly

In [6]:
import importlib.util
spec = importlib.util.spec_from_file_location("atmdata.py",
                                              "/Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/atmdata.py")
AtomDataModule = importlib.util.module_from_spec(spec)
spec.loader.exec_module(AtomDataModule)


{'Z': 26, 'fa': [11.7695, 7.3573, 3.5222, 2.3045], 'fb': [4.7611, 0.3072, 15.3535, 76.8805], 'fc': 1.0369}


In [7]:
print(AtomDataModule.XrayFF['Fe'])
print(AtomDataModule.XrayFF['Fe']['fa'])

{'Z': 26, 'fa': [11.7695, 7.3573, 3.5222, 2.3045], 'fb': [4.7611, 0.3072, 15.3535, 76.8805], 'fc': 1.0369}
[11.7695, 7.3573, 3.5222, 2.3045]


In [8]:
print(AtomDataModule.XrayFF['Fe']['fa'][1])

7.3573


In [50]:
# Cu radiation Ferrite 110 values from Jatczak
Theta= 44.60/2 # degrees
#Lambda=1.54178 # angstrom

#Values in GSASII
Lambda=1.5405 # angstrom, 8.0477 keV



sTL=math.sin(math.radians(Theta))/Lambda
print(sTL)

AtomicScatteringCoeff(AtomDataModule.XrayFF['Fe'],sTL)


0.24632012952223634


18.484212349418332

In [49]:
def AtomicScatteringCoeff(AtomDict, SinTheta_Lambda):
    FF=0
    for i in [0,1,2,3]:
        FF=FF+AtomDict['fa'][i]*math.exp(-AtomDict['fb'][i]*SinTheta_Lambda**2)
    FF=FF+AtomDict['fc']
    return FF

## Using FPCalc

In [53]:
sys.path.insert(0,'/Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/') # needed to "find" GSAS-II modules

import GSASIIscriptable as G2sc

# pick which element to get form factor (may come back as a list)
Atom=G2sc.G2elem.GetFormFactorCoeff('Fe')
Atom[0] # non ionized version

F=(G2sc.G2elem.ScatFac(Atom[0], (sTL**2))[0]) #[0] to return first term (out of one)



In [54]:
def LambdaAng2keV(Lambda):
    '''
    Convert between wavelength in angstroms and energy in keV
    '''
    keV=scipy.constants.h*scipy.constants.c/(Lambda*1e-10*scipy.constants.value("electron volt")*1000)
    return keV

In [55]:
LambdaAng2keV(Lambda)

8.04830888888025

In [56]:
Orbitals=G2sc.G2elem.GetXsectionCoeff('Fe')

LambdaAng2keV(Lambda)
#print(Orbitals)
(f_prime, f_2prime, mu)= G2sc.G2elem.FPcalc(Orbitals,LambdaAng2keV(Lambda))


In [58]:
print(F, f_prime, F+f_prime)

18.484212349418332 -1.1330686781563117 17.35114367126202
